In [25]:
from models.base_network import BaseNetwork, BaseNetwork1, EmbedLayer, FlattenLayer
from file_handling.load_datasets import load_mnist
from tqdm import tqdm_notebook as tqdm
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader

import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torchvision.models import resnet18
import torch.nn.functional as F
import torchvision.utils
import torchvision.models.resnet as resnet_utils

import numpy as np
from copy import deepcopy

import matplotlib.pyplot as plt

from preprocessing.noise_models import gaussian_noise, shot_noise, defocus_blur, zoom_blur, brightness, snow, clipped_zoom
import seaborn as sns
sns.set()

### Loading Datasets

In [14]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Lambda(lambda x: 255*x)])
test_data = datasets.CIFAR10(root='./cifar-10', train=False, download=False, transform=transform)
train_data = datasets.CIFAR10(root='./cifar-10', train=True, download=False, transform=transform)

In [15]:
train_loader = torch.utils.data.DataLoader(train_data, batch_size=16, shuffle=False, num_workers=2)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=1000, shuffle=False, num_workers=2)

### Initializing Networks

In [16]:
class MiniResNet(resnet_utils.ResNet):

    def __init__(self, block, layers, num_classes=10):
        super(resnet_utils.ResNet, self).__init__()
        self.inplanes = 64
        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3,
                               bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 64, layers[1], stride=1)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(64 * block.expansion, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
                
    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)

        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)

        return x

In [17]:
cross_criterion = nn.CrossEntropyLoss()
square_criterion = nn.modules.loss.MSELoss()

In [6]:
mini_resnet = BaseNetwork1(MiniResNet(resnet_utils.BasicBlock, [1,1]))

In [7]:
mini_resnet_opt = optim.RMSprop(mini_resnet.parameters(), lr=0.0005)

In [8]:
mini_resnet

BaseNetwork1(
  (0): MiniResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (layer2): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.

In [ ]:
# mini_resnet.train_model(epochs=1, print_every=2000, optimizer=mini_resnet_opt, 
#                         train_loader=train_loader, test_loader=test_loader, criterion=cross_criterion)

In [11]:
mini_resnet.load_model("saved_models/mini_reg_resnet.pt")

In [12]:
print(mini_resnet.test_model_once(noise_function=None, severity=0, test_loader=test_loader))
print(mini_resnet.test_model_once(noise_function=None, severity=0, test_loader=train_loader))

74.07
78.21


In [ ]:
mini_resnet.save_model("saved_models/mini_reg_resnet.pt")

In [18]:
trunc_mini_resnet = MiniResNet(resnet_utils.BasicBlock, [1,1])

In [19]:
trunc_mini_resnet(train_data[0][0][None,:,:,:])

tensor([[-0.3149,  0.0970, -1.3469, -0.3280,  0.7146,  0.6903, -0.7011,  0.4033,
          0.0488, -0.4237]], grad_fn=<ThAddmmBackward>)

In [20]:
trunc_mini_resnet = nn.Sequential(*list(trunc_mini_resnet.children())[:-1])
trunc_mini_resnet

Sequential(
  (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace)
  (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (4): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (5): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (con

In [21]:
mini_emb_resnet = BaseNetwork1(trunc_mini_resnet, FlattenLayer(), EmbedLayer(64, 8000), nn.Linear(8000, 10))
mini_emb_resnet

BaseNetwork1(
  (0): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (5): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_run

In [22]:
mini_emb_resnet_opt = optim.RMSprop(mini_emb_resnet.parameters(), lr=0.00005)

In [23]:
mini_emb_resnet.train_model(epochs=1, print_every=train_loader.batch_size, optimizer=mini_emb_resnet_opt, 
                        train_loader=train_loader, test_loader=test_loader, criterion=cross_criterion)

epoch: 1, iteration: 15, train acc: 0.10, loss: 2.319332
epoch: 1, iteration: 31, train acc: 0.11, loss: 2.313048
epoch: 1, iteration: 47, train acc: 0.09, loss: 2.297452


Process Process-3:
Process Process-4:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/Applications/anaconda/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/Applications/anaconda/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/Applications/anaconda/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/Applications/anaconda/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 96, in _worker_loop
    r = index_queue.get(timeout=MANAGER_STATUS_CHECK_INTERVAL)
  File "/Applications/anaconda/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/Applications/anaconda/lib/python3.6/multiprocessing/queues.py", line 104, in get
    if not self._poll(timeout):
  File "/Applications/anaconda/lib/python3.6/multiprocessing/connection.py", line 257, in po

KeyboardInterrupt: 

In [27]:
print(torch.__version__)

0.4.1


In [26]:
images, labels = iter(test_loader).next()
from plotting import visualization
# print(list(mini_emb_resnet.children()))
visualization.visualize_activations(images, labels, list(mini_emb_resnet.children())[-2], mini_emb_resnet)

AttributeError: 'Tensor' object has no attribute 'register_forward_hook'

In [ ]:
reg_net = BaseNetwork1(nn.Conv2d(3, 16, 3), nn.ReLU(), nn.MaxPool2d(2,2),
                        nn.Conv2d(16, 32, 3), nn.ReLU(), nn.MaxPool2d(2,2),
                        nn.Conv2d(32, 64, 3), nn.ReLU(), nn.MaxPool2d(2,2),
                        FlattenLayer(),
                        nn.Linear(256, 5000), nn.ReLU(),
                        nn.Linear(5000, 100), nn.ReLU(),
                        nn.Linear(100, 10))
# reg_net = BaseNetwork("reg_net", [(3, 16, 3),(16,32,3), (32, 64, 3),(256,5000),(5000,100),(100,10)],
#                                 ["conv", "conv", "conv", "fc", "fc", "fc"])
# emb_net = BaseNetwork("reg_net", [(3, 16, 5),(16,32,5), (800,5000),(5000,100),(100,10)],
#                                 ["conv", "conv", "fc", "fc", "fc"])
# reg_net = BaseNetwork("reg_net", [(3, 16, 5),(16,32,5),(800,100),(100,10)],
#                                 ["conv", "conv", "fc", "fc"])
# reg_net = BaseNetwork("reg_net", [(3*1024,5000),(5000,100),(100,10)],
#                                 ["fc", "fc", "fc"])
reg_net_opt = optim.RMSprop(reg_net.parameters(), lr=0.0005)

In [ ]:
reg_net

In [ ]:
emb_net = BaseNetwork1(nn.Conv2d(3, 16, 3), nn.ReLU(), nn.MaxPool2d(2,2),
                        nn.Conv2d(16, 32, 3), nn.ReLU(), nn.MaxPool2d(2,2),
                        nn.Conv2d(32, 64, 3), nn.ReLU(), nn.MaxPool2d(2,2),
                        FlattenLayer(),
                        EmbedLayer(256, 5000),
                        nn.Linear(5000, 100), nn.ReLU(),
                        nn.Linear(100, 10))
# emb_net = BaseNetwork("emb_net", [(3, 16, 3),(16,32,3), (32, 64, 3),(256,5000),(5000,100),(100,10)],
#                                 ["conv", "conv", "conv", "emb", "fc","fc"])
# emb_net = BaseNetwork("emb_net", [(3, 16, 5),(16,32,5), (800,5000),(5000,100),(100,10)],
#                                 ["conv", "conv", "emb", "fc", "fc"])
# emb_net = BaseNetwork("emb_net", [(3*1024,5000),(5000,100),(100,10)],
#                                 ["emb", "fc", "fc"])
emb_net_opt = optim.RMSprop(emb_net.parameters(), lr=0.000005, momentum=0.9)

In [ ]:
emb_net

### Training, Saving and Loading Models

In [ ]:
good_model.train_model(train_loader, 5, good_model_opt, criterion)

In [ ]:
reg_net.train_model(train_loader, 3, reg_net_opt, criterion)
reg_net.save_model("saved_models/reg_net_test.pt")

In [ ]:
emb_net.train_model(train_loader, 5, emb_net_opt, criterion)
emb_net.save_model("saved_models/emb_net_test.pt")

In [ ]:
reg_net.load_model("saved_models/reg_net_test.pt")
emb_net.load_model("saved_models/emb_net_test.pt")

### Testing Pure Accuracy on Test Set

In [ ]:
print("Test accuracy of REGULAR network with NO NOISE: {}".format(reg_net.test_model_once(test_loader, None, 0)))
print("Test accuracy of EMBEDDED network with NO NOISE: {}".format(emb_net.test_model_once(test_loader, None, 0)))

### Testing on Gaussian Noise

In [ ]:
print("Test accuracy of REGULAR network on GAUSSIAN NOISE: {}".format(reg_net.test_model(test_loader, gaussian_noise)))
print("Test accuracy of EMBEDDED network on GAUSSIAN NOISE: {}".format(emb_net.test_model(test_loader, gaussian_noise)))

In [ ]:
true = [57.19, 52.68, 45.93, 43.5, 40.63]
cool = [57.99, 56.6, 53.89, 52.77, 50.17]

plt.plot(np.r_[1:6], true)
plt.plot(np.r_[1:6], cool)
plt.legend(["Regular", "Embedding"])
plt.title("Model Accuracy on Gaussian Noise")
plt.xlabel("Noise Severity")
plt.xticks(range(1,6))
plt.ylabel("Accuracy")
plt.show()

### Testing on Shot Noise

In [ ]:
print("Test accuracy of REGULAR network on SHOT NOISE: {}".format(reg_net.test_model(test_loader, shot_noise)))
print("Test accuracy of EMBEDDED network on SHOT NOISE: {}".format(emb_net.test_model(test_loader, shot_noise)))

In [ ]:
true = [59.44, 57.59, 51.77, 47.88, 42.49]
cool = [58.48, 57.81, 55.65, 54.02, 51.42]

plt.plot(np.r_[1:6], true)
plt.plot(np.r_[1:6], cool)
plt.legend(["Regular", "Embedding"])
plt.title("Model Accuracy on Shot Noise")
plt.xlabel("Noise Severity")
plt.xticks(range(1,6))
plt.ylabel("Accuracy")
plt.show()

### Testing on Zoom Blur

In [ ]:
print("Test accuracy of REGULAR network on ZOOM BLUR: {}".format(reg_net.test_model(test_loader, zoom_blur)))
print("Test accuracy of EMBEDDED network on ZOOM BLUR: {}".format(emb_net.test_model(test_loader, zoom_blur)))

### Inspecting Convolution Weights

In [ ]:
weights_reg, bias_reg = deepcopy(reg_net.state_dict()['0.weight']), deepcopy(reg_net.state_dict()['0.bias'])
weights_emb, bias_emb = deepcopy(emb_net.state_dict()['0.weight']), deepcopy(emb_net.state_dict()['0.bias'])

In [ ]:
def norm(im):
    im = deepcopy(im)
    im = im - torch.min(im)
    im = im / torch.max(im)
    return im
plt.imshow(train_data[0][0].int().permute(1, 2 ,0))
plt.show()

In [ ]:
def imshow(img, to_norm=True, figsize=(15, 10)):
    img = deepcopy(img)
    if to_norm:
        img = norm(img)
    npimg = img.numpy()
    plt.figure(figsize=figsize)
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.axis("off")
    plt.show()
    
test_im,_ = train_data[0]
test_im = test_im[None,:,:,:]
test_im_reg = F.conv2d(test_im, weight=weights_reg, bias=bias_reg).permute(1,0,2,3)

for i in range(len(weights_reg)):
    weights_reg[i] = norm(weights_reg[i])
    test_im_reg[i] = norm(test_im_reg[i])

imshow(torchvision.utils.make_grid(train_data[0][0][:, None, :, :]))
imshow(torchvision.utils.make_grid(test_im_reg), to_norm=False)
imshow(torchvision.utils.make_grid(weights_reg), to_norm=False)




In [ ]:
test_im_emb = F.conv2d(test_im, weight=weights_emb, bias=bias_emb).permute(1,0,2,3)

# print(test_im_reg)

for i in range(len(weights_emb)):
    weights_emb[i] = norm(weights_emb[i])
    test_im_emb[i] = norm(test_im_emb[i])

imshow(torchvision.utils.make_grid(train_data[0][0][:, None, :, :]))
imshow(torchvision.utils.make_grid(test_im_emb), to_norm=False)
imshow(torchvision.utils.make_grid(weights_emb), to_norm=False)

In [ ]:
resnet_model = resnet18(pretrained=True)
resnet_model

In [ ]:
for param in resnet_model.parameters():
    param.requires_grad = False
# Replace the last fully-connected layer
# Parameters of newly constructed modules have requires_grad=True by default
trunc_resnet = nn.Sequential(*list(resnet_model.children())[:-3])

# trunc_resnet

In [ ]:
added_module = BaseNetwork1(FlattenLayer(), EmbedLayer(1024, 5000), nn.ReLU(), 
                             nn.Linear(5000, 10))

our_resnet = BaseNetwork1(trunc_resnet, added_module)

our_opt = optim.RMSprop(our_resnet.parameters(), lr=.0005)

In [ ]:
baseline = BaseNetwork1(trunc_resnet, FlattenLayer(), nn.ReLU(), nn.Linear(1024, 10))
base_opt = optim.Adagrad(baseline.parameters(), lr=.005)

In [ ]:
# baseline

In [ ]:
baseline.train_model(epochs=5, opt=base_opt, criterion=criterion, train_loader=train_loader)

In [ ]:
baseline.test_model_once(noise_function=None, severity=0, test_loader=test_loader)

In [ ]:
our_resnet.train_model(epochs=5, opt=our_opt, criterion=criterion, train_loader=train_loader)

In [ ]:
for epoch in range(5):
    run_loss = 0.0
    for i, data in enumerate(tqdm(train_loader)):
        inputs, labels = data
        base_opt.zero_grad()
        outputs = baseline(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        base_opt.step()
        run_loss += loss.item()
        if i % train_loader.batch_size == train_loader.batch_size - 1:
            print('[%d, %5d] loss: %.5f' %
                  (epoch + 1, i + 1, run_loss / (train_loader.batch_size)))
            run_loss = 0.0